In [19]:
from ultralytics import YOLO
import cv2
import time
import torch
import numpy as np

In [20]:
MODEL_PATH = "yolov8s.pt"

In [21]:
model = YOLO(MODEL_PATH)

100%|██████████| 21.5M/21.5M [00:00<00:00, 74.1MB/s]


In [22]:
if torch.cuda.is_available():
    model.to("cuda")
else:
    model.to("cpu")

In [23]:
camera = cv2.VideoCapture(0)

In [24]:
camera.release()

In [25]:
CLASS_NAMES = model.names

In [26]:
TARGET_CLASS_TRAFFIC_LIGHT = "traffic light"

In [13]:
camera = cv2.VideoCapture(0)
while True:
    ret, frame = camera.read()

    if not ret:
        print("Error while loading camera")
        break
    
    
    copyframe = frame.copy()
    results = model(frame, stream=False, verbose=False, conf=0.3)

    detected_traffic_lights = []

    for result in results:
        boxes = result.boxes
        for box in boxes:
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            class_name = CLASS_NAMES[cls]

            if class_name.lower() == TARGET_CLASS_TRAFFIC_LIGHT.lower():
                           
                roi = frame[y1:y2, x1:x2]

                hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

                avg_hue = np.mean(hsv[:,0,0])


                if avg_hue <= 10 or avg_hue >= 150:
                    light_color = "RED"
                    color = (0, 0, 255) # Red BGR
                elif 60 <= avg_hue <= 120:
                    light_color = "GREEN"
                    color = (0, 255, 0) # Green BGR
                else:
                    light_color = "UNKNOWN"
                    color = (0, 255, 255)

                detected_traffic_lights.append({
                    "box": (x1, y1, x2, y2),
                    "color": light_color,
                    "hue": avg_hue,
                    "confidence": conf,
                    "draw_color": color
                })


    for light in detected_traffic_lights:
             x1, y1, x2, y2 = light["box"]
             label = f"Light: {light['color']} ({light['hue']:.1f})"
             cv2.rectangle(copyframe, (x1, y1), (x2, y2), light["draw_color"], 2)
             cv2.putText(copyframe, label, (x1, y1 - 10),
                         cv2.FONT_HERSHEY_SIMPLEX, 0.5, light["draw_color"], 2)


    cv2.imshow("detected", copyframe)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()



            




In [27]:
import cv2
import numpy as np # Make sure numpy is imported

# --- Configuration ---
# Define HSV color ranges (adjust these based on your lighting conditions/camera)
# OpenCV Hue range is 0-179
RED_RANGES = [
    # Range 1 (covers lower hues)
    {'lower': np.array([0, 0, 100]), 'upper': np.array([20, 255, 255])},
    # Range 2 (covers higher hues, wrapping around 180)
    {'lower': np.array([160, 0, 50]), 'upper': np.array([179, 255, 255])}
]
GREEN_RANGE = {'lower': np.array([35, 0, 50]), 'upper': np.array([90, 255, 255])}
# Optional: Add Yellow/Amber
YELLOW_RANGE = {'lower': np.array([20, 80, 100]), 'upper': np.array([80, 255, 255])}

TARGET_CLASS_TRAFFIC_LIGHT = "traffic light"

# --- Main Loop ---
camera = cv2.VideoCapture(0)
if not camera.isOpened():
    print("Error: Could not open camera.")
    exit()

while True:
    ret, frame = camera.read()

    if not ret:
        print("Error while loading frame from camera")
        break

    copyframe = frame.copy()
    # Use the model to detect objects
    # Consider adding classes=[index_of_traffic_light] if you only want that class
    results = model(frame, stream=False, verbose=False, conf=0.3)

    detected_traffic_lights = []

    for result in results:
        boxes = result.boxes
        for box in boxes:
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            x1, y1, x2, y2 = map(int, box.xyxy[0])

            # Ensure CLASS_NAMES exists and cls is a valid key
            if cls not in CLASS_NAMES:
                print(f"Warning: Class index {cls} not found in CLASS_NAMES.")
                continue

            class_name = CLASS_NAMES[cls]

            # Check if the detected object is a traffic light
            if class_name.lower() == TARGET_CLASS_TRAFFIC_LIGHT.lower():
                # Extract the Region of Interest (ROI)
                roi = frame[y1:y2, x1:x2]

                # --- Improved Color Detection ---
                if roi.size == 0: # Check if ROI is empty
                    print("Warning: Detected traffic light ROI is empty.")
                    continue

                hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

                # Calculate pixel counts for each color
                red_pixels = 0
                for r in RED_RANGES:
                    mask = cv2.inRange(hsv_roi, r['lower'], r['upper'])
                    red_pixels += cv2.countNonZero(mask)

                green_mask = cv2.inRange(hsv_roi, GREEN_RANGE['lower'], GREEN_RANGE['upper'])
                green_pixels = cv2.countNonZero(green_mask)

                # Optional: Yellow/Amber
                yellow_mask = cv2.inRange(hsv_roi, YELLOW_RANGE['lower'], YELLOW_RANGE['upper'])
                yellow_pixels = cv2.countNonZero(yellow_mask)

                # Determine the dominant color based on pixel count
                # Add more conditions if including yellow/other colors
                light_color = "UNKNOWN"
                draw_color = (0, 255, 255) # Yellow BGR for Unknown

                # Add a threshold: require a minimum number of pixels to avoid noise
                # This threshold might need tuning based on ROI size
                min_pixels_threshold = (roi.shape[0] * roi.shape[1]) * 0.05 # e.g., 5% of ROI area

                if red_pixels > green_pixels and red_pixels > min_pixels_threshold:
                    light_color = "RED"
                    draw_color = (0, 0, 255) # Red BGR
                elif green_pixels > red_pixels and green_pixels > min_pixels_threshold:
                    light_color = "GREEN"
                    draw_color = (0, 255, 0) # Green BGR
                elif yellow_pixels > red_pixels and yellow_pixels > green_pixels and yellow_pixels > min_pixels_threshold:
                    light_color = "YELLOW"
                    draw_color = (0, 127, 255) # Orange BGR

                # Store detected light info
                detected_traffic_lights.append({
                    "box": (x1, y1, x2, y2),
                    "color": light_color,
                    "confidence": conf,
                    "draw_color": draw_color
                    # Removed 'hue' as it's less meaningful now
                })


    # --- Drawing ---
    for light in detected_traffic_lights:
             x1, y1, x2, y2 = light["box"]
             # Corrected label formatting
             label = f"{light['color']} ({light['confidence']:.2f})" # Show color and confidence
             cv2.rectangle(copyframe, (x1, y1), (x2, y2), light["draw_color"], 2)
             # Put text slightly above the box
             label_y = y1 - 10 if y1 - 10 > 10 else y1 + 10 # Adjust if too close to top
             cv2.putText(copyframe, label, (x1, label_y),
                         cv2.FONT_HERSHEY_SIMPLEX, 0.6, light["draw_color"], 2)


    # Display the resulting frame
    cv2.imshow("Traffic Light Detection", copyframe)

    # Exit loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
camera.release()
cv2.destroyAllWindows()


In [18]:
import cv2
import numpy as np
import sys

# --- Global variable to store the latest frame ---
# We need this so the mouse callback can access the frame
# where the click happened.
current_frame = None
hsv_value_clicked = None

# --- Mouse Callback Function ---
def get_hsv_on_click(event, x, y, flags, param):
    global current_frame, hsv_value_clicked
    # Check if the event is a left mouse button click
    if event == cv2.EVENT_LBUTTONDOWN and current_frame is not None:
        # Ensure coordinates are within frame bounds
        if 0 <= y < current_frame.shape[0] and 0 <= x < current_frame.shape[1]:
            # Get the BGR pixel value at the clicked coordinate (y, x)
            bgr_pixel = current_frame[y, x]

            # Convert the single BGR pixel to HSV
            # Note: Need to reshape to (1, 1, 3) for cvtColor
            hsv_pixel = cv2.cvtColor(np.uint8([[bgr_pixel]]), cv2.COLOR_BGR2HSV)[0][0]

            hsv_value_clicked = hsv_pixel # Store the value
            print(f"Clicked at ({x}, {y}) - BGR: {bgr_pixel}, HSV: {hsv_pixel}")
        else:
            print(f"Clicked outside frame bounds at ({x}, {y})")


# --- Main Program ---

# 1. Initialize Video Capture
camera = cv2.VideoCapture(0) # 0 is usually the default webcam

if not camera.isOpened():
    print("Error: Could not open webcam.")
    sys.exit()

# 2. Create a window and set the mouse callback
window_name = "Webcam Feed - Click to get HSV"
cv2.namedWindow(window_name)
cv2.setMouseCallback(window_name, get_hsv_on_click)

print("Webcam opened. Click on the window to get HSV values. Press 'q' to quit.")

while True:
    # 3. Read a frame from the webcam
    ret, frame = camera.read()

    if not ret:
        print("Error: Failed to grab frame.")
        break

    # Store the frame globally for the callback function
    current_frame = frame.copy() # Use copy to avoid issues if frame is modified later

    # 4. Display the HSV value on the frame (optional)
    if hsv_value_clicked is not None:
        hsv_text = f"Last HSV: {hsv_value_clicked}"
        cv2.putText(current_frame, hsv_text, (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)

    # 5. Show the frame in the window
    cv2.imshow(window_name, current_frame)

    # 6. Check for exit key ('q')
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        print("Exiting...")
        break

# 7. Release resources
camera.release()
cv2.destroyAllWindows()


Webcam opened. Click on the window to get HSV values. Press 'q' to quit.
Clicked at (237, 274) - BGR: [ 80  92 117], HSV: [ 10  81 117]
Clicked at (592, 345) - BGR: [76 62 64], HSV: [124  47  76]
Clicked at (337, 309) - BGR: [51 46 61], HSV: [170  63  61]
Clicked at (433, 318) - BGR: [60 50 54], HSV: [132  43  60]
Clicked at (216, 195) - BGR: [89 70 73], HSV: [125  54  89]
Clicked at (110, 390) - BGR: [163 163 163], HSV: [  0   0 163]
Clicked at (36, 456) - BGR: [ 72 117  85], HSV: [ 51  98 117]
Clicked at (357, 448) - BGR: [68 45 45], HSV: [120  86  68]
Exiting...
